***Total: 42 points***

Complete this homework by writing R code to complete the following tasks. Keep in mind:

i. Empty chunks have been included where code is required
ii. For Problem 2e, you should include a image (screen shot), which can be loaded into R
iii. This homework requires use of data files:

  - `BRCA.genome_wide_snp_6_broad_Level_3_scna.seg` (Problems 1, 2)
  - `GIAB_highconf_v.3.3.2.vcf.gz` (Problem 3)
  
iv. You will be graded on your code and output results (knitted .html file). The assignment is worth 42 points total; partial credit can be awarded.

This assignment is due on **Dec 1, 2022**. 

For additional resources, please refer to these links:  
Problems 1 & 2:  
  - https://www.bioconductor.org/packages/devel/bioc/vignettes/plyranges/inst/doc/an-introduction.html
  - https://bioconductor.org/packages/release/bioc/vignettes/GenomicRanges/inst/doc/GenomicRangesIntroduction.html  
Problem 3:  
  - https://bioconductor.org/packages/release/bioc/vignettes/Rsamtools/inst/doc/Rsamtools-Overview.pdf  
Problem 4: 
  - https://bioconductor.org/packages/release/bioc/vignettes/VariantAnnotation/inst/doc/VariantAnnotation.pdf  

# Problem 1: Overlaps between genomic regions and copy number alterations. (14 points total)

### Preparation
Load copy number segment results as shown in *2.1 BED format* of *Lecture16_GenomicData.Rmd*. You will use the same file as in the lecture notes, `BRCA.genome_wide_snp_6_broad_Level_3_scna.seg`. Here is code to get you started.

In [1]:
#load packages
suppressPackageStartupMessages({
    library(tidyverse)
    library(GenomicRanges)
    library(plyranges)
    library(VariantAnnotation)
})

In [2]:
#old wdir
getwd()

#new wdir
data_dir = '/workspaces/tfcb_2022/homeworks/homework07'
setwd(data_dir)
getwd()

[1] "/workspaces/tfcb_2022/homeworks/homework07"

[1] "/workspaces/tfcb_2022/homeworks/homework07"

In [8]:
list.files()

[1] "BRCA_IDC_cfDNA.bam"                           
[2] "BRCA_IDC_cfDNA.bam.bai"                       
[3] "BRCA.genome_wide_snp_6_broad_Level_3_scna.seg"
[4] "GIAB_highconf_v.3.3.2.vcf.gz"                 
[5] "GIAB_highconf_v.3.3.2.vcf.gz.tbi"             
[6] "Homework07.ipynb"                             
[7] "homework07.Rmd"                               
[8] "hs37d5.fa"                                    
[9] "README.md"

In [9]:
segs <- read.delim("BRCA.genome_wide_snp_6_broad_Level_3_scna.seg", as.is = TRUE)
mode(segs$Chromosome) <- "character" 
segs[segs$Chromosome == 23, "Chromosome"] <- "X"
segs.gr <- as(segs, "GRanges")

### a. Find the segments in `segs.gr` that have *any* overlap with the region `chr8:128,746,347-128,755,810` (4 points)
Print out the first five unique TCGA IDs.

In [10]:
#myRange variable for GRanges with sequence location of interest
myGRange <- GRanges(seqnames = "8", ranges = IRanges(start = 128746347, end = 128755810))
#overlap of sequences with the Grange and convert to dataframe
segs.overlap <- find_overlaps(segs.gr, myGRange) %>%
    as.data.frame()
#print the first 5 unique (TCGA) IDs
print (segs.overlap$Sample[1:5])

[1] "TCGA-3C-AAAU-10A-01D-A41E-01" "TCGA-3C-AAAU-01A-11D-A41E-01"
[3] "TCGA-3C-AALI-10A-01D-A41E-01" "TCGA-3C-AALI-01A-11D-A41E-01"
[5] "TCGA-3C-AALJ-10A-01D-A41E-01"


### b. Find the mean of the `Segment_Mean` values for copy number segments that have *any* overlap with the region chr17:37,842,337-37,886,915. (4 points)

In [11]:
#myRange variable for GRanges with sequence location of interest, new variable name
myGRange_1b <- GRanges(seqnames = "17", ranges = IRanges(start = 37842337, end = 37886915))
#overlap of sequences with the Grange and convert to dataframe
segs.overlap_1b <- find_overlaps(segs.gr, myGRange_1b) %>%
    as.data.frame() 
#mean of all segment mean values from segs.overlap_1b 
mean(segs.overlap_1b$Segment_Mean)

[1] 0.1420197

### c. Find the patient sample distribution of copy number for `PIK3CA` (hg19). (6 points)
Find the counts of samples with deletion (D; `Segment_Mean < -0.3`), neutral (N; `Segment_Mean >= -0.3 & Segment_Mean <= 0.3`), gain (G; `Segment_Mean > 0.3`) segments that have `any` overlap with `PIK3CA` gene coordinates.  


In [13]:
#myRange variable for GRanges with PIK3CA gene sequence location
myGRange_PIK3CA <- GRanges(seqnames = "3", ranges = IRanges(start = 179148126, end = 179240093))
#overlap of sequences with the Grange and convert to dataframe
segs.overlap_PIK3CA <- find_overlaps(segs.gr, myGRange_PIK3CA) %>%
    as.data.frame()

#filter segs.overlap_PIK3CA for Segment_mean less than -0.3 for dels
deletion_seqs <- segs.overlap_PIK3CA %>%
    filter(Segment_Mean < -0.3)
#count samples with deletions by counting the rows
del <- nrow(deletion_seqs)
sprintf('The counts of samples with deletion segments is %s.', del) #sprintf function

#filter segs.overlap_PIK3CA for Segment_mean between -0.3 and 0.3 for neutral segs
neutral_seqs <- segs.overlap_PIK3CA %>%
    filter(Segment_Mean >= -0.3 & Segment_Mean <= 0.3)
#count samples with deletions by counting the rows
neutral <- nrow(neutral_seqs)
sprintf('The counts of samples with neutral segments is %s.', neutral)

#filter segs.overlap_PIK3CA for Segment_mean greater than 0.3 to find gain segments
gain_seqs <- segs.overlap_PIK3CA %>%
    filter(Segment_Mean > 0.3)
#count samples with deletions by counting the rows
gain <- nrow(gain_seqs)
sprintf('The counts of samples with gain segments is %s.', gain)

ERROR: Error in parse(text = x, srcfile = src): <text>:12:61: unexpected ','
11: del <- nrow(deletion_seqs)
12: print(('The counts of samples with deletion segments is %s.',
                                                                ^


# Problem 2: Frequency of copy number alteration events within genomic regions. (12 points total) 

This problem will continue to use the copy number data stored in `segs.gr`.

### a. Create a genome-wide tile of 1Mb windows for the human genome (`hg19`). (4 points)
See *3.1 Tiling the genome* of *Lecture16_GenomicData.Rmd* for hints.


### b. Find the 1Mb window with the most frequent overlapping deletions. (4 points)
Find the 1Mb windows with `any` overlap with deletion copy number segments. Assume a deletion segment is defined as a segment in `segs.gr` having `Segment_Mean < -0.3`. 

Return one of the 1Mb window `Granges` entry with the highest frequency (count) of deletion segments.

Hint: Subset the `segs.gr` to only rows with `Segment_Mean < -0.3`. 

### c. Visually inspect the deletion overlap result from part (b) using IGV. (4 points)
Provide a screen shot of IGV at the 1Mb window with the most frequent overlap with deletion segments. The image should include the segments from `BRCA.genome_wide_snp_6_broad_Level_3_scna.seg` loaded. 

In [ ]:
#save screenshot and load into R with png package (see: https://stackoverflow.com/questions/23861000/displaying-images-in-r-in-version-3-1-0)
#install.packages('png') #install if necessary 
library(png)
pp <- readPNG("myfile.png")
plot.new() 
rasterImage(pp,0,0,1,1)

# Problem 3: Reading and annotating genomic variants (16 points total)

### Preparation

In [6]:
vcfFile <- "GIAB_highconf_v.3.3.2.vcf.gz"

### a. Load variant data from VCF file `GIAB_highconf_v.3.3.2.vcf.gz` for `chr8:128,700,000-129,000,000`. (4 points)
Note: use genome build `hg19`.

### b. Combine the fields of the VCF genotype information into a table. (4 points)
You may use your choice of data objects (e.g. `data.frame`).

### c. Retrieve the following information at chr8:128747953. (8 points)
Print out the SNP ID (i.e. "rs ID"), reference base (`REF`), alterate base (`ALT`), genotype (`GT`), depth (`DP`), allele depth (`ADALL`), phase set (`PS`).

Hints: 

  i. `REF` and `ALT` are in the output of `rowRanges(vcf)`. See Section `3a` in `Lecture16_VariantCalls.ipynb` 
  ii. To get the sequence of `DNAString`, use `as.character(x)`.  
  ii. To get the sequence of `DNAStringSet`, use `as.character(unlist(x))`. 
  iii. To expand a list of information for `geno`, use `unlist(x)`.  

  